Imports

In [1]:
#to be able to acess google drive
from google.colab import drive
drive.mount('/content/drive')

#premade modules
get_ipython().system('pip install pytorch-gradcam')

import cv2
import random
import PIL
from torchvision.utils import make_grid, save_image
from gradcam.utils import visualize_cam
from gradcam import GradCAM, GradCAMpp
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import transforms
import torchvision.models as models
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
import os
import time
import sklearn.metrics
from itertools import *

Mounted at /content/drive
     |████████████████████████████████| 6.0MB 4.7MB/s 
  Created wheel for pytorch-gradcam: filename=pytorch_gradcam-0.2.1-cp36-none-any.whl size=5270 sha256=0ba1b2d6ae8cd28a61381b212d2ca6e1439d801d26ef49e26301f43adc5ac836
  Stored in directory: /root/.cache/pip/wheels/e8/1e/35/d24150a078a90ce0ad093586814d4665e945466baa89907300
Successfully built pytorch-gradcam


Paths

In [2]:
#these are the only global variables
base_dir = "/content/drive/My Drive/Colab Notebooks"
imgs_dir = os.path.join(base_dir, "Images")
imgs_COVID_dir = os.path.join(imgs_dir, "CT_COVID")
imgs_NonCOVID_dir = os.path.join(imgs_dir, "CT_NonCOVID")
data_split_dir = os.path.join(base_dir,"Data-split")
ds_COVID_dir = os.path.join(data_split_dir,"COVID")
ds_NonCOVID_dir = os.path.join(data_split_dir,"NonCOVID")
code_dir = os.path.join(base_dir, "Code")
Buffer_dir = os.path.join(code_dir, "Buffer")

Data preparation

In [25]:
normalize_grayscale = transforms.Normalize(mean=[0.449], std=[0.226])
normalize_rgb = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

def getTrainTransform_rgb():

    trainTransform = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop((224),scale=(0.5,1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize_rgb
    ])

    return trainTransform, "trainTransform RGB"

def getValTransformCrop():

    trainTransform = transforms.Compose([
        transforms.Resize(256),
        transforms.RandomResizedCrop((224),scale=(0.5,1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize_rgb
    ])

    return trainTransform, "trainTransform RGB"

def getTrainTransform_grayscale():

    trainTransform = transforms.Compose([
        #transforms.Resize( (224,224) ),
        transforms.Resize(256),
        transforms.RandomResizedCrop((224),scale=(0.5,1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize_grayscale
    ])

    return trainTransform, "trainTransform grayscale"

def getTrainTransform_newSize():

    trainTransform = transforms.Compose([
        transforms.Resize( (407,287) ),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize_rgb
    ])

def getValTransform_rgb():

    valTransform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize_rgb
    ])

    return valTransform, "valTransform RGB"

def getValTransform_grayscale():

  valTransform = transforms.Compose([
      transforms.Resize(256),
      transforms.CenterCrop(224),
      transforms.ToTensor(),
      normalize_grayscale
  ])

  return valTransform, "valTransform grayscale"

In [5]:
def read_txt(txt_path):
    with open(txt_path) as f:
        lines = f.readlines()
    txt_data = [line.strip() for line in lines]
    return txt_data

class CovidCTDataset(Dataset):
    def __init__(self, txt_COVID, txt_NonCOVID, transform=None, color = "RGB", n_crops = 1):

        self.color = color
        self.n_crops = n_crops
        self.txt_path = [txt_COVID,txt_NonCOVID]
        self.classes = ['CT_COVID', 'CT_NonCOVID']
        self.num_cls = len(self.classes)
        
        #List of lists. Each list contains the full path of an image and 0 if Covid, 1 if non Covid
        self.img_list = []
        
        for c in range(self.num_cls):
            cls_list = [[item, c] for item in read_txt(self.txt_path[c])]
            self.img_list += cls_list
            
        self.transform = transform

        
    #aparrently __len__() and __getitem__() must be implemented for map-style datasets    
    def __len__(self):
        return len(self.img_list)

    def __getitem__(self, idx):

      img_path = self.img_list[idx][0]

      if (self.color == "grayscale"):
        original_img = Image.open(img_path).convert('L')
      else:
        original_img = Image.open(img_path).convert('RGB')

      crops = []
      for i in range(self.n_crops):
        #Appends a different crop of the same image, as long as there is randomness in the transform
        crops.append(self.transform(original_img))

      sample = {'img': crops,
                'label': int(self.img_list[idx][1])}
      
      #Returns dictionary that contains image (torch tensor, because of transform) and the label (0,1)
      return sample

def getDataloaders(trainTransform, valTransform, batchsize, color, n_crops):

    trainset = CovidCTDataset(txt_COVID = os.path.join(ds_COVID_dir, "trainCT_COVID.txt"),
                              txt_NonCOVID = os.path.join(ds_NonCOVID_dir, "trainCT_NonCOVID.txt"),
                              transform = trainTransform, color = color)

    testset = CovidCTDataset(txt_COVID = os.path.join(ds_COVID_dir, "testCT_COVID.txt"),
                             txt_NonCOVID = os.path.join(ds_NonCOVID_dir, "testCT_NonCOVID.txt"),
                             transform= valTransform, color = color, n_crops = n_crops)

    #I wonder what difference it makes to change drop_last.
    train_loader = DataLoader(trainset, batch_size=batchsize, drop_last=False, shuffle=True)
    test_loader = DataLoader(testset, batch_size=batchsize, drop_last=False, shuffle=False)

    return train_loader, test_loader

K-fold validation

In [6]:
class Patient:
    def __init__(self, ID, imgs, covid):
        self.ID = ID
        self.imgs = imgs
        self.covid = covid
        
    def __len__(self):
        return len(self.imgs)
    
    def __str__(self):
        return f"Patient ID: {self.ID}\nImages: {self.imgs}"

class Partition:
    def __init__(self,size):
        self.size = size
        self.patients = []
        self.occupied = 0
        
    def add(self, patient):
        self.patients.append(patient)
        self.occupied += len(patient)
    
    def freeSpace(self):
        return (self.size - self.occupied)
    
    def __len__(self):
        return self.size
    
    def __str__(self):
        string = f"Size: {self.size}\n"
        string += f"Occupied: {self.occupied}\n"
        string += f"Patients: {self.patients}"
        return string

class RandomPartitions:
    def __init__(self, patients, nPartitions, proportions = []):
        
        self.partitionList = []
        
        #Calculates total number of images
        nImages = np.sum([ len(patient) for patient in patients ])
        
        #equal proportions
        if len(proportions) == 0:
            proportions = (1/nPartitions) * np.ones(nPartitions)
        else:
            proportions = np.array(proportions)

        #caculates partitions size and converts to int
        partitionSizes = (np.round(nImages*proportions)).astype(int)
        #makes sure that sum of all partition sizes is equal to number of images
        partitionSizes[-1] = nImages - partitionSizes[0:nPartitions-1].sum()

        #Adds to the partition list, a partition for every partition size
        for partitionSize in partitionSizes:
            self.partitionList.append(Partition(partitionSize))
        
        priorityQueue = sorted(patients,key=len).copy()        

        #A number for each partition
        choices = np.arange(nPartitions)
        
        #while there is something to allocate
        while(len(priorityQueue) > 0):
            
            #extracts patient from priority Queue
            patient = priorityQueue.pop()
            
            #probability of being allocated to each partition
            p = np.array([partition.freeSpace() for partition in self.partitionList])
            p = p/np.sum(p)
            
            #randomly chooses a partition
            partition = np.random.choice(choices,p = p)
            
            #puts patient into partition
            self.partitionList[partition].add(patient)
    
    def toLists(self):
        #gets a list of lists from a list of partition objets
        partitions = [partition.patients for partition in self.partitionList]
        
        return partitions
            
    def __len__(self):
        return len(self.partitionList)

#Organizes images by patients
def loadPatients():
    
    #Relevant paths
    info_dir = os.path.join(base_dir, "Info")
    COVID_info = os.path.join(info_dir,"COVID-CT-MetaInfo.csv")
    NonCOVID_info = os.path.join(info_dir,"NonCOVID-CT-MetaInfo.csv")

    COVID_dict = dict() #key is the patient and the value is a list of image paths
    NonCOVID_dict = dict()
    
    #COVID
    file = open(COVID_info,"r")

    #skips first line (its useless for us)
    file.readline()

    #Goes through evey line
    for line in file:

        #Retrieves the information we need
        image_name, patient = line.split(",")[0:2]

        image_path = os.path.join(imgs_COVID_dir, image_name)

        #patient already exists
        if(patient in COVID_dict):
            COVID_dict[patient].append(image_path)

        #patient does not exist
        else:
            COVID_dict[patient] = [image_path]

    file.close()

    #NonCOVID
    file = open(NonCOVID_info,"r")

    #skips first line
    file.readline()

    #Goes through evey line
    for line in file:

        #Retrieves the information we need
        image_name, patient = line.split(",")[1:3]
        patient = patient.rstrip("\n")

        image_path = os.path.join(imgs_NonCOVID_dir, image_name)

        #Patient already exists
        if(patient in NonCOVID_dict):
            NonCOVID_dict[patient].append(image_path)

        #Patient does not exist
        else:
            NonCOVID_dict[patient] = [image_path]

    file.close()

    #Converts dictionnary into list of Patient objects
    patients = []
    for key,value in COVID_dict.items():
        patients.append( Patient(key,value,1) )
        
    for key,value in NonCOVID_dict.items():
        patients.append( Patient(key,value,0) )

    #Returns a list of patient objects
    return patients

#converts lists of lists of an element into a list of all elements
def unpack(outerList):
    newList = []
    for inerList in outerList:
        newList += inerList
    return newList

class CovidCTDataset_patients(Dataset):
  def __init__(self, patients, transform, color = "RGB", n_crops = 1):
      
    self.color = color
    self.n_crops = n_crops
    self.img_list = []
    self.transform = transform
    
    #unpacks list patients
    for patient in patients: #for every patient
        for img in patient.imgs: #for every image of the patient
            self.img_list.append( (img , patient.covid) ) #adds (image,label) to img_list 
      
  def __len__(self):
    return len(self.img_list)

  def __getitem__(self, idx):

    img_path = self.img_list[idx][0]
    if (self.color == "grayscale"):
      original_img = Image.open(img_path).convert('L')
    else:
      original_img = Image.open(img_path).convert('RGB')

    crops = []
    for i in range(self.n_crops):
      #Appends a different crop of the same image, as long as there is randomness in the transform
      crops.append(self.transform(original_img))

    sample = {'img': crops,
              'label': int(self.img_list[idx][1])}
    
    return sample

#produces dataloaders for simple hold out validation (train + test)
def holdOut_getLoaders(train_test_split, trainTransform, testTransform, batchsize, color, n_crops):

  #gets a list of all the patients from the data
  allPatients = loadPatients()
  
  train_test_proportions = [train_test_split, 1 - train_test_split]

  #train+val and test split
  train_list, test_list = RandomPartitions(allPatients, 2, train_test_proportions).toLists()

  train_set = CovidCTDataset_patients(train_list, transform = trainTransform, color = color)
  test_set = CovidCTDataset_patients(test_list, transform = testTransform, color = color, n_crops = n_crops)

  train_loader = DataLoader(train_set, batch_size=batchsize, drop_last=False, shuffle=True)
  test_loader = DataLoader(test_set, batch_size=batchsize, drop_last=False, shuffle=False)

  return train_loader, test_loader

def kfold_getLists(trainVal_test_split,k):

  #gets a list of all the patients from the data
  allPatients = loadPatients()
  
  trainVal_test_proportions = [trainVal_test_split, 1 - trainVal_test_split]

  #train+val and test split
  trainVal_list, test_list = RandomPartitions(allPatients, 2, trainVal_test_proportions).toLists()

  #devides trainVal_list into k equal partitions
  kfold_partitions = RandomPartitions(trainVal_list,k).toLists()

  #creates the different combinations of train and validation sets
  
  #each element is (train_list, val_list)
  trainAndVal_lists = []
  
  for i, partition in enumerate(kfold_partitions):

    val_list = partition
    train_list = unpack( kfold_partitions[:i] + kfold_partitions[i+1:] )

    trainAndVal_lists.append( (train_list, val_list) )

  return trainAndVal_lists, test_list

def kfold_lists2loaders(trainAndVal_lists, test_list, train_transform, val_transform, batchsize, color,  n_crops = 1):

    test_set = CovidCTDataset_patients(test_list, transform = val_transform,color = color)
    test_loader = DataLoader(test_set, batch_size=batchsize, drop_last=False, shuffle=False)

    trainAndValLoaders = []
    
    for  (train_list, val_list) in trainAndVal_lists:

        train_set = CovidCTDataset_patients(train_list, train_transform,color)
        val_set = CovidCTDataset_patients(val_list, val_transform, color, n_crops)

        train_loader = DataLoader(train_set, batch_size=batchsize, drop_last=False, shuffle=True)
        val_loader = DataLoader(val_set, batch_size=batchsize, drop_last=False, shuffle=False)

        trainAndValLoaders.append( (train_loader, val_loader) )

    return trainAndValLoaders, test_loader

#combines kfold_getLists and kfold_lists2loaders
def kfold_getDataloaders(train_transform, val_transform, trainVal_test_split, k, batchsize, color, n_crops = 1):
    
    trainAndVal_lists, test_list = kfold_getLists(trainVal_test_split,k)
    trainAndValLoaders, test_loader = kfold_lists2loaders(trainAndVal_lists, test_list, train_transform, val_transform, batchsize, 
                                                          color, n_crops = 1)

    return trainAndValLoaders, test_loader

def kfold_completeTrain(device, trainAndValLoaders, getModel, getOptimizer, epochs, learning_rate, weight_decay = 0, momentum = 0):

    kfold_metrics = []

    for i, (train_loader, val_loader) in enumerate(trainAndValLoaders):

        print(f"\nFold: {i}\n")

        model = 0
        
        del model
        
        #getModel returns model and modelname, we only want the model here
        model = getModel()[0]

        fold_metrics = completeTrain(device, model, getOptimizer, train_loader, val_loader, epochs, learning_rate, weight_decay)

        kfold_metrics.append(fold_metrics)
    
    return getAverageMetrics_kfold(kfold_metrics)

def getAverageMetrics_kfold(kfold_metrics):
    
    epochs = len(kfold_metrics[0][1])
    k = len(kfold_metrics)

    CM_train_average = np.zeros( (epochs,2,2), dtype = np.float32 )
    CM_val_average = np.zeros( (epochs,2,2 ), dtype = np.float32 )
    LOSS_train_average = np.zeros(epochs, dtype = np.float32)
    LOSS_val_average = np.zeros(epochs, dtype = np.float32)

    for CM_train, LOSS_train, CM_val, LOSS_val in kfold_metrics:

        CM_train_average += CM_train
        CM_val_average += CM_val
        LOSS_train_average += LOSS_train
        LOSS_val_average += LOSS_val

    CM_train_average /= k
    CM_val_average /= k
    LOSS_train_average /= k
    LOSS_val_average /= k

    return CM_train_average, LOSS_train_average, CM_val_average, LOSS_val_average

Models

In [ ]:
def setUpGetResnet50(classifier_widths, pretrained = True, unfrozenLayers = 0, classifier_input_size = 2048):

  class Classifier(nn.Module):
    def __init__(self, classifier_widths):
      super(Classifier, self).__init__()
      #classifier module
      self.layers = nn.Sequential()

      #hidden layers
      for i in range(1,len(classifier_widths)):
        self.layers.add_module(f"{ 3*(i-1) }",nn.Linear(classifier_widths[i-1],classifier_widths[i]))
        self.layers.add_module(f"{ 3*(i-1) + 1 }",nn.ReLU(True))
        self.layers.add_module(f"{ 3*(i-1) + 2 }",nn.Dropout())

      #output layer
      self.layers.add_module(f"{3*( len(classifier_widths) - 1) }",nn.Linear(classifier_widths[-1],2))

    def forward(self, x):        
        return self.layers(x)

  #for first layer
  classifier_widths.insert(0, classifier_input_size)

  name = f"resnet-50, pretrained = {pretrained}, number of unfrozen layers = {unfrozenLayers}, width of classifier layers = {classifier_widths}"

  def getResnet50():
    
    model = models.resnet50(pretrained = pretrained)

    if pretrained:
      #freeze all parameters
      for params in model.parameters():
        params.requires_grad = False
      
      #unfreeze some from the end
      parameters = list(model.parameters())
      parameters.reverse()
      for i in range(2*unfrozenLayers):
        parameters[i].requires_grad = True

    model.fc = Classifier(classifier_widths)

    return model, name

  return getResnet50

In [ ]:
def setUpGetDensenet169(classifier_widths, pretrained = True, unfrozenLayers = 0, classifier_input_size = 1664):
  
  class Classifier(nn.Module):
    def __init__(self, classifier_widths):
      super(Classifier, self).__init__()
      #classifier module
      self.layers = nn.Sequential()

      #hidden layers
      for i in range(1,len(classifier_widths)):
        self.layers.add_module(f"{ 3*(i-1) }",nn.Linear(classifier_widths[i-1],classifier_widths[i]))
        self.layers.add_module(f"{ 3*(i-1) + 1 }",nn.ReLU(True))
        self.layers.add_module(f"{ 3*(i-1) + 2 }",nn.Dropout())

      #output layer
      self.layers.add_module(f"{3*( len(classifier_widths) - 1) }",nn.Linear(classifier_widths[-1],2))

    def forward(self, x):        
        return self.layers(x)

  #for first layer
  classifier_widths.insert(0, classifier_input_size)

  name = f"densenet, pretrained = {pretrained}, number of unfrozen layers = {unfrozenLayers}, width of classifier layers = {classifier_widths}"

  def getDensenet169():
    
    model = models.densenet169(pretrained = pretrained)

    if pretrained:
      for params in model.parameters():
        params.requires_grad = False
      
      parameters = list(model.parameters())
      parameters.reverse()
      for i in range(2*unfrozenLayers):
        parameters[i].requires_grad = True

    model.classifier = Classifier(classifier_widths)

    return model, name

  return getDensenet169

In [ ]:
def setUpGetVgg16(classifier_widths, unfrozenLayers = 0, classifier_input_size = 25088):

  class nothing(nn.Module):
      def __init__(self):
          super(nothing, self).__init__()        
      def forward(self, x):        
          return x

  class Classifier(nn.Module):
    def __init__(self, classifier_widths):
      super(Classifier, self).__init__()
      #classifier module
      self.layers = nn.Sequential()

      #hidden layers
      for i in range(1,len(classifier_widths)):
        self.layers.add_module(f"{ 3*(i-1) }",nn.Linear(classifier_widths[i-1],classifier_widths[i]))
        self.layers.add_module(f"{ 3*(i-1) + 1 }",nn.ReLU(True))
        self.layers.add_module(f"{ 3*(i-1) + 2 }",nn.Dropout())

      #output layer
      self.layers.add_module(f"{3*( len(classifier_widths) - 1) }",nn.Linear(classifier_widths[-1],2))

    def forward(self, x):        
        return self.layers(x)

  #for first layer
  classifier_widths.insert(0,classifier_input_size)

  #this needs to be adepted as a function of the paramerters
  modelname = f"vgg16 pretained, number of unfrozen layers = {unfrozenLayers}, width of classifier layers = {classifier_widths}"

  def getVgg16():

    model = models.vgg16(pretrained=True)

    #Freezes every layer
    for i,param in enumerate(model.parameters()):
        #there is a *2 because every layer has two parameter tensors, one for weights and one for biases
        if i > (25 - 2*unfrozenLayers):
          break
        param.requires_grad = False

    #the average pool only does something if the input image size is different than 224x224
    model.avgpool = nothing()

    model.classifier = Classifier(classifier_widths)

    return model, modelname
  
  return getVgg16

def genGetVgg16DiffClass(base):
    
  widths = [ [ int(base/2) ], [base], [2*base] ]
  
  for w in widths.copy():
    widths.append( w + [ int(w[0]/2) ] )
      
  for w in islice(widths.copy(), 3, None):
    widths.append( w + [ int(w[1]/2) ])

  getVgg16s = []
  for width in widths:
    getVgg16s.append( setUpGetVgg16(classifier_widths=width) )
  
  return getVgg16s

In [ ]:
def setUpGetVgg16_grayscale(unfrozenLayers = 0, classifier_input_size = 25088, classifier_widths = [1000], convert_type = "average"):

  class nothing(nn.Module):
      def __init__(self):
          super(nothing, self).__init__()        
      def forward(self, x):        
          return x

  class Classifier(nn.Module):
    def __init__(self, classifier_widths):
      super(Classifier, self).__init__()
      #classifier module
      self.layers = nn.Sequential()

      #hidden layers
      for i in range(1,len(classifier_widths)):
        self.layers.add_module(f"{ 3*(i-1) }",nn.Linear(classifier_widths[i-1],classifier_widths[i]))
        self.layers.add_module(f"{ 3*(i-1) + 1 }",nn.ReLU(True))
        self.layers.add_module(f"{ 3*(i-1) + 2 }",nn.Dropout())

      #output layer
      self.layers.add_module(f"{3*( len(classifier_widths) - 1) }",nn.Linear(classifier_widths[-1],2))

    def forward(self, x):        
        return self.layers(x)

  #for first layer
  classifier_widths.insert(0,classifier_input_size)

  #this needs to be adepted as a function of the paramerters
  modelname = f"vgg16 pretained, number of unfrozen layers = {unfrozenLayers}, width of classifier layers = {classifier_widths}, convert = {convert_type}"

  def getVgg16():

    model = get_vgg16_grayscale(convert_type)

    #Freezes every layer
    for i,param in enumerate(model.parameters()):
        #there is a *2 because every layer has two parameter tensors, one for weights and one for biases
        if i > (25 - 2*unfrozenLayers):
          break
        param.requires_grad = False

    #the average pool only does something if the input image size is different than 224x224
    model.avgpool = nothing()

    model.classifier = Classifier(classifier_widths)

    return model, modelname
  
  return getVgg16

def get_vgg16_grayscale(convert_type):
  model = models.vgg16(pretrained=True)

  #get old first conv layer
  old_input_layer = model.features[0]

  #gets old first conv layer weights
  rgb_weights = old_input_layer.weight.data

  if (convert_type == "luma"):
    rgb_weights[:,0,:,:] =  0.897*rgb_weights[:,0,:,:]
    rgb_weights[:,1,:,:] =  1.761*rgb_weights[:,1,:,:]
    rgb_weights[:,2,:,:] =  0.342*rgb_weights[:,2,:,:]

  #calculates new grayscale weights
  grayscale_weights = rgb_weights.sum(dim=1)
  #adds a dummy dimension
  grayscale_weights = grayscale_weights.view(64,1,3,3)

  #new conv_layer 
  new_input_layer = nn.Conv2d(1, 64, (3, 3), stride=(1, 1), padding=(1, 1))
  #copys the grayscale weights to new conv layer
  new_input_layer.weight.data = grayscale_weights
  #copys old bias to new conv layer
  new_input_layer.bias = old_input_layer.bias

  #replaces old conv layer with new conv layer
  model.features[0] = new_input_layer

  return model

In [ ]:
def printOutputShape(model):
  patients = loadPatients()
  dataset = CovidCTDataset_patients(patients, getTrainTransform_newSize()[0])
  loader = DataLoader(dataset, batch_size=1)

  batch = iter(loader).next()
  input = batch["img"][0]

  with torch.no_grad():
    output = model(input)

  print(output.shape)

printOutputShape(model)

optimizers

In [ ]:
def getRMSprop():
    return torch.optim.RMSprop, "RMSprop"

def getSGD():
    return torch.optim.SGD, "SGD"

def getAdam():
    return torch.optim.Adam, "Adam"

Train and validation functions

In [ ]:
def train(device, model, dataloader, criterion, optimizer):
        
    model.train()
    
    #gets data size
    data_size = len(dataloader.dataset)
    
    #all targets of epoch
    epoch_target = torch.zeros(data_size)
    #all predictions of epoch
    epoch_pred = torch.zeros(data_size)
    
    epoch_loss = 0
    
    #Goes through every batch
    for i, batch in enumerate(dataloader):
                        
        #Separates input and output
        #I must use [0] because it is always a list, even if there only one element
        imgs, target = batch['img'][0].to(device), batch['label'].to(device)
        #gets batch size
        batch_size = len(target)
                        
        #Forward pass
        output = model(imgs)
        
        #Calculates loss for batch
        loss = criterion(output, target)
                        
        #Adds batch loss to epoch_loss
        epoch_loss += loss.item()*batch_size
        
        #Calculates gradient (backpropagation)
        optimizer.zero_grad()
        loss.backward()
        
        #updates weights
        optimizer.step()
        
        #Gets prediction from class scores
        pred = output.argmax(dim=1)
        
        #groups data of all batches
        epoch_pred[i*batch_size : (i + 1) * batch_size] = pred
        epoch_target[i*batch_size : (i + 1) * batch_size] = target
    
    #creates confusion matrix
    CM = sklearn.metrics.confusion_matrix(epoch_target, epoch_pred)

    #loss per sample
    mean_epoch_loss = epoch_loss/data_size
    
    return CM, mean_epoch_loss

def val(device, model, dataloader, criterion):
            
    model.eval()
    
    #gets size of data
    data_size = len(dataloader.dataset)
    
    #Stores data for the entire epoch
    epoch_pred = torch.zeros(data_size)
    epoch_target = torch.zeros(data_size)
    
    #sum of the loss of every batch/not used
    epoch_loss = 0
     
    #Does not keep track of operations for gradient calculation
    with torch.no_grad():
                        
        for i, batch in enumerate(dataloader):
            
            #Seperates input and output
            imgs, target = batch['img'], batch['label'].to(device)
            #gets size of batch
            batch_size = len(target)

            output = torch.zeros( (batch_size,2), device = torch.device(device) )

            #goes through every crop         
            for crop in imgs:

                crop = crop.to(device)
                #passes crop though the model
                output += model(crop)

            #average score of all crops
            output = output / len(imgs)
            
            #sums batch loss to epoch_loss
            epoch_loss += criterion(output, target).item()*batch_size
            
            #Calculates prediction based on class scores
            pred = output.argmax(dim=1)
            
            #groups data of all batches
            epoch_target[i*batch_size : (i + 1)*batch_size] = target
            epoch_pred[i*batch_size : (i + 1)*batch_size] = pred
        
        CM = sklearn.metrics.confusion_matrix(epoch_target, epoch_pred)
        
        #loss per sample
        mean_epoch_loss = epoch_loss/data_size
    
    return CM, mean_epoch_loss

Main training loop

In [ ]:
def completeTrain(device, model, getOptimizer, train_loader, val_loader, epochs, learning_rate, weight_decay = 0, momentum = 0):

    torch.cuda.empty_cache()
    model = model.to(device)

    #get optimizer returns a function and a name, here we only want the function. That is why there is [0]
    optimizer = getOptimizer()[0](model.parameters(), learning_rate, weight_decay = weight_decay, momentum = momentum)
    
    criterion = torch.nn.CrossEntropyLoss()

    LOSS_train, CM_train, LOSS_val, CM_val = [], [], [], []

    t_total = 0

    for epoch in range(epochs): 
    
        print(f"Epoch: {epoch:2.0f},", end = " ")
            
        t_start = time.time()

        cm_val, loss_val = val(device, model,val_loader,criterion)
        
        t_end = time.time()
        t_val = t_end-t_start
        print(f"Val time: {t_val:5.2f}s", end = " ")

        t_start = time.time()

        cm_train, loss_train = train(device, model, train_loader,criterion, optimizer)

        t_end = time.time()
        t_train = t_end-t_start
        print(f"Train time: {t_train:5.2f}s")
            
        CM_train.append(cm_train)
        LOSS_train.append(loss_train)
        CM_val.append(cm_val)
        LOSS_val.append(loss_val)

        t_total += (t_train + t_val)
    
    print(f"\nTotal time: {formatTime(t_total)}")

    return np.array(CM_train), np.array(LOSS_train), np.array(CM_val), np.array(LOSS_val)

Hyperparameter search

In [ ]:
class hyperparameter():

  next_id = 0

  def __init__(self, batchsize, getTrainTransform, getValTransform, getModel, getOptimizer, learning_rate, 
              epochs, weight_decay, momentum, color, n_crops, trainVal_test_split = None, k = None):
      
    self.id = hyperparameter.next_id
    hyperparameter.next_id += 1

    self.trainVal_test_split = trainVal_test_split
    self.k = k
    self.batchsize = batchsize
    self.getTrainTransform = getTrainTransform
    self.getValTransform = getValTransform
    self.getModel = getModel
    self.getOptimizer = getOptimizer
    self.learning_rate = learning_rate
    self.epochs = epochs
    self.weight_decay = weight_decay
    self.momentum = momentum
    self.color = color
    self.n_crops = n_crops

  def getId(self):
      return str(self.id)

  def __str__(self):
    return f"Model {self.id}, trainVal_test_split = {self.trainVal_test_split}, k = {self.k}, batchsize = {self.batchsize}, train transform = {self.getTrainTransform()[1]}, val transform = {self.getValTransform()[1]}, model = {self.getModel()[1]}, optimizer = {self.getOptimizer()[1]}, learning rate = {self.learning_rate}, epochs = {self.epochs}, weight decay = {self.weight_decay}, momentum = {self.momentum}, color = {self.color}, n_crops = {self.n_crops}"

def addParameterCombinations(prev_combinations, parameter_list):
    
    new_combinations = []
    
    for combination in prev_combinations:
        
        for parameter in parameter_list:
            
            new_combination = combination.copy() + [parameter]
            new_combinations.append(new_combination)
            
    return new_combinations

def getParamCombinations(parameters):
    
    for i, parameter in enumerate(parameters):

      if (i == 0):
        combinations = [ [p] for p in parameter]
      else:
        combinations = addParameterCombinations(combinations, parameter)

    hps = []
    for c in combinations:
      hps.append( hyperparameter(c[0],c[1],c[2],c[3],c[4],c[5],c[6],c[7],c[8],c[9],c[10]) )

    return hps

#used as sorting key later on
def get2ele(Tuple): #gets second element of a tuple
    return Tuple[1]

def configs2file(filename, configs):

    #creates the file if it does not exist
    #erases file and creates new file if already exists
    f = open(filename, "w")
    f.close()

    #opens file for appending
    f = open(filename, "a")

    for (name, mcc, acc, time) in configs:
        f.write(f"[mcc {mcc:2.2f}, acc {acc:2.2f}, time {time}] : {name}\n\n")
    f.close()

def loadNpz(filename):
    filename = os.path.join(Buffer_dir, filename + ".npz")

    npzfile = np.load(filename)

    CM_train = npzfile["CM_train"]
    LOSS_train = npzfile["LOSS_train"]
    CM_val = npzfile["CM_val"] 
    LOSS_val = npzfile["LOSS_val"]

    return CM_train, LOSS_train, CM_val, LOSS_val

def hyperparameterSetSearch(hps):

    dev = "cuda"
    trainVal_test_split = 0.9
    k = 5

    trainAndVal_lists, test_list = kfold_getLists(trainVal_test_split = trainVal_test_split, k = k)

    configs = []

    for hp in hps:

        hp.trainVal_test_split = trainVal_test_split
        hp.k = k

        t_start = time.time()

        #I dont need the test loader here, that is why there is a [0] at the end
        trainAndValLoaders = kfold_lists2loaders(trainAndVal_lists, test_list, hp.getTrainTransform()[0], hp.getValTransform()[0], hp.batchsize, hp.color, hp.n_crops )[0]

        CM_train, LOSS_train, CM_val, LOSS_val = kfold_completeTrain(dev, trainAndValLoaders, hp.getModel, hp.getOptimizer, hp.epochs, hp.learning_rate, hp.weight_decay, hp.momentum)

        #creates a path to save metrics for current combination
        filename = os.path.join(Buffer_dir, f"Model {hp.getId()}" )

        #saves these metrics to a file
        np.savez(filename, CM_train = CM_train, LOSS_train = LOSS_train, CM_val = CM_val, LOSS_val = LOSS_val)

        #gets only the accuracy metric
        metrics = getMetrics(CM_val)

        MCC_val = metrics["mcc"]
        ACC_val = metrics["accuracy"]

        t_end = time.time()
        t = t_end - t_start

        configs.append( ( str(hp), maMax(MCC_val)[1] , maMax(ACC_val)[1] , formatTime(t) ) ) 

        #sorts list by val accuracy
        configs.sort(key = get2ele, reverse = True)

        #saves configs to a text file
        configs2file( os.path.join(Buffer_dir, "configs.txt") , configs)

def hyperparameterGridSearch(batchsize, getTrainTransform, getValTransform, getModel, getOptimizer, learning_rate, epochs, 
                             weight_decay, momentum, color, n_crops):

    #adds the parameter lists to a list
    parameters = [batchsize, getTrainTransform, getValTransform, getModel, getOptimizer, learning_rate, epochs, weight_decay, 
                  momentum, color, n_crops]

    #gets all of the different combinations of the parameters
    hps = getParamCombinations(parameters)
    
    #prints information
    print(f"Number of combinations: {len(hps)}\n")
    
    hyperparameterSetSearch(hps)

def printConfigsFile():
  path = os.path.join(Buffer_dir,"configs.txt")
  f = open(path)
  print(f.read())
  f.close()

def printCurrentConfig(model):
  path = os.path.join(Buffer_dir,"configs.txt")
  f = open(path)

  for line in f:
    if(line != "\n"):
      split1 = line.split(":")
      split2 = split1[1].split(",")[0].strip()
      if model == split2:
        print(line, end ="")

In [ ]:
def CM_get_elements(CM):
    
  TN = CM[:,0,0]
  FP = CM[:,0,1]
  FN = CM[:,1,0]
  TP = CM[:,1,1]
  
  return TN, FP, FN, TP

def CM_get_normalized_elements(CM):
  
  TN, FP, FN, TP = CM_get_elements(CM)

  #Number of real positives in the data
  P = TP + FN
  #number of real negatives in the data
  N = TN + FP

  TN = TN/N
  FP = FP/N
  FN = FN/P
  TP = TP/P

  return TN, FP, FN, TP

#caculates accuracy, precision and recall from confusion matrix
def getMetrics(CM):

  TN, FP, FN, TP =  CM_get_elements(CM)
    
  #accuracy
  ACC = (TP + TN) / (TP + TN + FP + FN)
  #Precision
  P = TP / (TP + FP)
  #Recall
  R = TP / (TP + FN)
  #F1 score
  F1 = 2 * ((P*R)/(P + R))
  #Matthews correlation coefficient
  MCC = ((TP * TN) - (FP * FN))/np.sqrt( (TP + FP)*(TP + FN)*(TN + FP)*(TN + FN) )
  
  #adds metrics to a dictionary
  metrics = { "accuracy":ACC, "precision":P, "recall":R, "f1":F1, "mcc":MCC }

  return metrics

#calculates the moving average of a vector
def movingAverage(vec):
    
    new_vec = np.zeros(len(vec))
    
    new_vec[0] = (2*vec[0] + vec[1])/3
    new_vec[-1] = (2*vec[-1] + vec[-2])/3
    
    for i in range(1, len(vec) - 1):
        new_vec[i] = (vec[i-1] + vec[i] + vec[i + 1])/3
    
    return new_vec

#prints the max of the moving average
def maMax(vec):

    ma_vec = movingAverage(vec)

    maMax_i = np.argmax(ma_vec)
    maMax_value = ma_vec[maMax_i]

    return (maMax_i + 1) , maMax_value
    
def plotMetrics(curves, labels, name_y, ymin = None, ymax = None, moving_average = True):
  
  epochs = np.arange(1, len(curves[0]) + 1)
  
  if moving_average:
    for i in range(len(curves)):
        curves[i] = movingAverage(curves[i])

  fig, ax = plt.subplots()
  ax.grid()
  if ( (ymin != None) and (ymax != None) ):
    ax.set_ylim([ymin,ymax])
  ax.set_xlabel("Epochs")
  ax.set_title(name_y)

  for c,label in zip(curves,labels):
    ax.plot(epochs, c, label = label)

  plt.legend()
  plt.tight_layout()
  plt.show()

def plotConfusionMatrix(CM_train, CM_val, mv = True):

  TN_train, FP_train, FN_train, TP_train = CM_get_normalized_elements(CM_train)
  TN_val, FP_val, FN_val, TP_val = CM_get_normalized_elements(CM_val)

  metrics = [[TN_train,TN_val,"TN"], [FP_train, FP_val,"FP"], [FN_train, FN_val, "FN"],[TP_train, TP_val, "TP"]]

  for i in range(len(metrics)):
    metrics[i][0] = movingAverage(metrics[i][0])
    metrics[i][1] = movingAverage(metrics[i][1])

  epochs = np.arange(1 , CM_train.shape[0] + 1 )
  
  fig, axes = plt.subplots(2,2)
  axes = axes.flatten()

  axes[0].set_ylim([0.5,1])
  axes[1].set_ylim([0,0.5])
  axes[2].set_ylim([0,0.5])
  axes[3].set_ylim([0.5,1])
  
  fig.suptitle("Confusion matrix")
  for metric, ax in zip(metrics,axes):

    ax.grid()
    ax.set_xlabel("Epochs")
    ax.set_title(metric[2])
    
    ax.plot(epochs,metric[0],label = "Train")
    ax.plot(epochs,metric[1],label = "Val")

  plt.legend()
  plt.tight_layout()
  plt.show()

def plotAcc_kfoldVal(kfold_metrics):

    fig, (axTrain, axVal) = plt.subplots(2)

    axTrain.set_title("Train accurary")
    axVal.set_title("Validation accurary")

    for i, kfold_metric in enumerate(kfold_metrics):

        [ (CM_train, LOSS_train), (CM_val,LOSS_val) ] = kfold_metric

        ACC_train = getMetrics(CM_train)["accuracy"]
        ACC_val = getMetrics(CM_val)["accuracy"]

        axTrain.plot(movingAverage(ACC_train), label = f"fold {i}")
        axVal.plot(movingAverage(ACC_val), label = f"fold {i}")
    
    plt.tight_layout()

    plt.show()

def displayAllMetrics(CM_train, LOSS_train, CM_val, LOSS_val):

    val_acc = getMetrics(CM_val)["accuracy"]

    print(f"Maximum validation accuracy: {maMax(val_acc)[1]:.2f} at epoch {maMax(val_acc)[0]}")

    #gets all metrics
    trainMetrics = getMetrics(CM_train)
    valMetrics = getMetrics(CM_val)
    
    #extracts individual metrics
    ACC_train = trainMetrics["accuracy"]
    P_train = trainMetrics["precision"]
    R_train = trainMetrics["recall"]
    MCC_train = trainMetrics["mcc"]
    
    ACC_val = valMetrics["accuracy"]
    P_val = valMetrics["precision"]
    R_val = valMetrics["recall"]
    MCC_val = valMetrics["mcc"]

    #difference between validation loss and training loss
    LOSS_diff = LOSS_val - LOSS_train

    plotMetrics([MCC_train, MCC_val], ["Train", "Val"],"MCC", ymin = 0, ymax = 1) #MCC
    plotMetrics([ACC_train,ACC_val],["Train", "Val"],"Accuracy",ymin = 0.5, ymax = 1) # accuracy
    plotMetrics([LOSS_train,LOSS_val],["Train", "Val"],"Loss") #loss
    plotMetrics([LOSS_diff],["Loss differenece"],"Loss differenece") #loss difference
    print(f"Area under loss difference curve: {LOSS_diff.sum():.2f}")
    plotMetrics([P_train,P_val],["Train", "Val"],"Precision",ymin = 0.5, ymax = 1) #Precision
    plotMetrics([R_train,R_val],["Train", "Val"],"Recall",ymin = 0.5, ymax = 1) #Recall
    plotConfusionMatrix(CM_train, CM_val)

def formatTime(seconds):

    hours = np.floor( seconds / (60*60) )
    seconds_left = seconds % (60*60)
    minutes = np.floor(seconds_left / 60)
    seconds = seconds % 60

    return (f"{hours:.0f}h {minutes:.0f}m {seconds:.2f}s")

In [ ]:
def getImgs(device, loader):

  denormalize = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                    std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                    std = [ 1., 1., 1. ]),
                              ])

  dataset = loader.dataset

  #indexes = np.random.randint( len(dataset), size = 4 )
  indexes = [1,10,20,30]
  imgs = []
  labels = []

  for i in indexes:
    sample = dataset[i]
    img = sample["img"][0]

    labels.append(sample["label"])

    torch_img = denormalize(img).to(device)
    normed_torch_img = img.view(1,3,224,224).to(device)

    imgs.append( (torch_img,normed_torch_img) )
  
  return imgs, labels

def grad_cam2(indexes, device, loader, model, model_type, layer_name):

  denormalize = transforms.Compose([ transforms.Normalize(mean = [ 0., 0., 0. ],
                                                      std = [ 1/0.229, 1/0.224, 1/0.225 ]),
                                  transforms.Normalize(mean = [ -0.485, -0.456, -0.406 ],
                                                      std = [ 1., 1., 1. ]),
                                ])

  dataset = loader.dataset

  imgs = []
  labels = []

  for i in indexes:
    sample = dataset[i]
    img = sample["img"][0]

    labels.append(sample["label"])

    torch_img = denormalize(img).to(device)
    normed_torch_img = img.view(1,3,224,224).to(device)

    imgs.append( (torch_img,normed_torch_img) )

  configs = [
      dict(model_type=model_type, arch=model, layer_name=layer_name)
  ]

  for config in configs:
      config['arch'].to(device).eval()

  cams = [
      [cls.from_config(**config) for cls in (GradCAM, GradCAMpp)]
      for config in configs
  ]

  results = []
  for gradcam, gradcam_pp in cams:
    for torch_img, normed_torch_img in imgs:
      mask, _ = gradcam(normed_torch_img)
      heatmap, result = visualize_cam(mask, torch_img)

      results.append(result)

  fig, axes = plt.subplots( len(imgs), 2, figsize = (15,30))

  for i in range(len(imgs)):

    original_img = transforms.ToPILImage()( imgs[i][0].cpu() )
    result = transforms.ToPILImage()(results[i])
    axes[i][0].imshow(original_img, aspect='auto')
    axes[i][1].imshow(result,aspect='auto')
    
    if labels[i] == 0:
      correct_class = "Non COVID"
    else:
      correct_class = "COVID"

    normed_torch_img = imgs[i][1]
    output = model(normed_torch_img).argmax(dim = 1)
    
    if output == 0:
      predicted_class = "Non COVID"
    else:
      predicted_class = "COVID"

    axes[i][0].set_title("Bonne classe: " + correct_class, fontsize = 25)
    axes[i][1].set_title("Classe estimée: " + predicted_class, fontsize = 25)

    axes[i][0].axis("off")
    axes[i][1].axis("off")

  plt.subplots_adjust(wspace=0)

In [ ]:
def grad_cam(device, loader, trained_model):

  #turns requires_grad on for all of the layers
  for params in trained_model.parameters():
    params.requires_grad = True

  class densenet_grad_cam(nn.Module):
    def __init__(self, trained_model):
      super(densenet_grad_cam, self).__init__()
      
      # get the trained network
      self.densenet = trained_model
      
      # disect the network to access its last convolutional layer
      self.features_conv = self.densenet.features
      
      # add the average global pool
      self.global_avg_pool = nn.AvgPool2d(kernel_size=7, stride=1)
      
      # get the classifier of the vgg19
      self.classifier = self.densenet.classifier
      
      # placeholder for the gradients
      self.gradients = None
    
    # hook for the gradients of the activations
    def activations_hook(self, grad):
      self.gradients = grad
        
    def forward(self, x):
      x = self.features_conv(x)
      
      # register the hook
      h = x.register_hook(self.activations_hook)
      
      # don't forget the pooling
      x = self.global_avg_pool(x)
      x = x.view((1, 1664))
      x = self.classifier(x)
      return x
    
    def get_activations_gradient(self):
      return self.gradients
    
    def get_activations(self, x):
      return self.features_conv(x)

  #geting images
  imgs, labels = getImgs("cuda", loader)

  results = []
  for torch_img, normed_torch_img in imgs:

    model = densenet_grad_cam(trained_model = trained_model).to(device)
    # set the evaluation mode
    model.eval()

    torch_img = torch_img.cpu().permute(1,2,0).numpy()

    # get the most likely prediction of the model
    scores = model(normed_torch_img)
    pred = int(scores.argmax(dim = 1))

    # get the gradient of the output with respect to the parameters of the model
    scores[:, pred].backward()

    # pull the gradients out of the model
    gradients = model.get_activations_gradient()

    # pool the gradients across the channels
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])

    # get the activations of the last convolutional layer
    activations = model.get_activations(normed_torch_img).detach()

    # weight the channels by corresponding gradients
    for i in range(512):
        activations[:, i, :, :] *= pooled_gradients[i]
        
    # average the channels of the activations
    heatmap = torch.mean(activations, dim=1).squeeze().cpu().numpy()

    # relu on top of the heatmap
    # expression (2) in https://arxiv.org/pdf/1610.02391.pdf
    heatmap = np.maximum(heatmap, 0)

    # normalize the heatmap
    heatmap /= np.max(heatmap)

    heatmap = cv2.resize(heatmap, torch_img.shape[0:2])

    heatmap = np.uint8(255 * heatmap)
    torch_img = np.uint8(255 * torch_img)

    heatmap = cv2.cvtColor(cv2.applyColorMap(heatmap, cv2.COLORMAP_JET), cv2.COLOR_BGR2RGB)

    superimposed_img = 0.4* heatmap + torch_img
    superimposed_img /= np.max(superimposed_img)

    results.append(superimposed_img)

  fig, axes = plt.subplots( len(imgs), 2, figsize = (15,30))

  for i in range(len(imgs)):
    original_img = transforms.ToPILImage()( imgs[i][0].cpu() )
    axes[i][0].imshow(original_img, aspect='auto')
    axes[i][1].imshow(results[i],aspect='auto')
    
    if labels[i] == 0:
      correct_class = "Non COVID"
    else:
      correct_class = "COVID"

    normed_torch_img = imgs[i][1]
    output = model(normed_torch_img).argmax(dim = 1)
    
    if output == 0:
      predicted_class = "Non COVID"
    else:
      predicted_class = "COVID"

    axes[i][0].set_title("Bonne classe: " + correct_class, fontsize = 25)
    axes[i][1].set_title("Classe estimée: " + predicted_class, fontsize = 25)

    axes[i][0].axis("off")
    axes[i][1].axis("off")

  plt.subplots_adjust(wspace=0)

Hold out validation call

In [ ]:
train_loader, test_loader = getDataloaders(getTrainTransform_rgb()[0], getValTransform_rgb()[0], batchsize = 16, color = "RGB", 
                                           n_crops = 1)

In [ ]:
#gets an instance of the model
getVGG16 = setUpGetVgg16(unfrozenLayers = 4, classifier_widths=[100])
vgg16, modelname = getVGG16()

CM_train, LOSS_train, CM_val, LOSS_val = completeTrain("cuda", vgg16, getRMSprop, train_loader, test_loader, epochs = 5, learning_rate = 0.00001, weight_decay = 0.01, momentum = 0)

In [ ]:
displayAllMetrics(CM_train, LOSS_train, CM_val, LOSS_val)

In [ ]:
#gets an instance of the model
getResnet = setUpGetResnet50(unfrozenLayers = 4, classifier_widths=[256])
resnet, modelname = getResnet()

CM_train, LOSS_train, CM_val, LOSS_val = completeTrain("cuda", resnet, getRMSprop, train_loader, test_loader, epochs = 12, learning_rate = 0.00001, weight_decay = 0.01, momentum = 0)

In [ ]:
displayAllMetrics(CM_train, LOSS_train, CM_val, LOSS_val)

In [ ]:
#gets an instance of the model
getDensenet = setUpGetDensenet169(unfrozenLayers = 4, classifier_widths=[200])
densenet, modelname = getDensenet()

CM_train, LOSS_train, CM_val, LOSS_val = completeTrain("cuda", densenet, getRMSprop, train_loader, test_loader, epochs = 10, learning_rate = 0.0001, weight_decay = 0.001, momentum = 0)

In [ ]:
displayAllMetrics(CM_train, LOSS_train, CM_val, LOSS_val)

In [ ]:
indexes = [33,58,61,56]

grad_cam2(indexes,"cuda", test_loader, vgg16, model_type = "vgg", layer_name = "features_29")
grad_cam2(indexes,"cuda", test_loader, resnet, model_type = "resnet", layer_name = "layer4")
grad_cam2(indexes,"cuda", test_loader, densenet, model_type = "densenet", layer_name = "features_norm5")

Kfold validation call

In [ ]:
trainAndValLoaders, test_loader = kfold_getDataloaders(getTrainTransform_rgb()[0], getTrainTransform_rgb()[0], trainVal_test_split = 0.9, k = 5, 
                                                       batchsize = 16, n_crops = 3)

CM_train, LOSS_train, CM_val, LOSS_val = kfold_completeTrain("cuda", trainAndValLoaders, getVgg16, getRMSprop, epochs = 15, learning_rate = 0.00001, 
                                                             weight_decay = 0.01, momentum = 0 )

In [ ]:
displayAllMetrics(CM_train, LOSS_train, CM_val, LOSS_val)

hyperparameter grid search

In [ ]:
getResnets = []

getResnets.append(setUpGetResnet50(unfrozenLayers=2, classifier_widths=[256]))

hyperparameterGridSearch(batchsize = [16],
                         getTrainTransform = [getTrainTransform_rgb],
                         getValTransform = [getTrainTransform_rgb],
                         getModel = getResnets,
                         getOptimizer = [getRMSprop],
                         learning_rate = [0.00001], 
                         epochs = [30],
                         weight_decay = [0.001], 
                         momentum = [0],
                         color = ["RGB"],
                         n_crops = [3])

Hyperparameter set search

In [ ]:
hp1 = hyperparameter(batchsize = 16, 
                     getTrainTransform = getTrainTransform_rgb, 
                     getValTransform = getTrainTransform_rgb, 
                     getModel = setUpGetDensenet169(unfrozenLayers=4, classifier_widths=[128]),
                     getOptimizer = getRMSprop,
                     learning_rate = 0.0001,
                     epochs = 30,
                     weight_decay = 0.001,
                     momentum = 0,
                     color = "RGB",
                     n_crops = 3)

hyperparameters = [hp1]

hyperparameterSetSearch(hyperparameters)


Displaying metrics saved to file

In [ ]:
printConfigsFile()

In [ ]:
name = "Model 13"

CM_train, LOSS_train, CM_val, LOSS_val = loadNpz(name)

printCurrentConfig(name)
displayAllMetrics(CM_train, LOSS_train, CM_val, LOSS_val)

In [ ]:
p = 24

Metrics = getMetrics(CM_val)

acc = Metrics["accuracy"][p]
mcc = Metrics["mcc"][p]
precision = Metrics["precision"][p]
recall = Metrics["recall"][p]

print(f"acc: {acc:.2f}")
print(f"mcc: {mcc:.2f}")
print(f"precision: {precision:.2f}")
print(f"recall: {recall:.2f}")

TN, FP, FN, TP = CM_get_normalized_elements(CM_val)

print()
print(f"TN:{TN[p]:.2f}")
print(f"FP:{FP[p]:.2f}")
print(f"FN:{FN[p]:.2f}")
print(f"TP:{TP[p]:.2f}")

print()
print(f"Loss: {LOSS_val[p]:.2f}")
print(f"Loss diff: {LOSS_val[p] - LOSS_train[p]:.2f}")